In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import torch

from sentence_transformers import util
from emma import Vectorizer

## Работа с векторизатором

Векторизацию входного текста, содержащего математические выражения, выполняет `Vectorizer`:

In [4]:
vectorizer = Vectorizer("basic-go/math-ru-sbert")

Чтобы получить векторное представление фрагмента текста, необходимо использовать метод `encode` модели векторных представлений:

In [5]:
half_transcription = "одна вторая"

half_emb = vectorizer.encode(half_transcription)

Числовой вектор, представляющий фрагмент текста, имеет размерность 768:

In [6]:
half_emb.shape

(1, 768)

В следующем примере выражения `uni_expr` и `tex_expr` определяют одно и тоже математическое выражение, причём их векторные представления обладают высокой мерой косинусного сходства. Вместе с тем выражения `tex_expr` и `another_tex_expr` различаются, а отвечающая им мера косинусного сходства имеет меньшее значение.

In [7]:
uni_expr = "1/α + 1/β = 1/γ"
uni_emb = vectorizer.encode(uni_expr)

tex_expr = r"\(\frac{1}{\alpha} + \frac{1}{\beta} = \frac{1}{\gamma}\)"
tex_emb = vectorizer.encode(tex_expr)

another_tex_expr = r"\(\alpha + \beta = \gamma\)"
another_tex_emb = vectorizer.encode(another_tex_expr)

print(util.cos_sim(uni_emb, tex_emb))
print(util.cos_sim(uni_emb, another_tex_emb))

tensor([[0.9027]])
tensor([[0.7251]])


Приведём пример, демонстрирующий возможность осуществления поиска. Создадим документы для поискового индекса:

In [8]:
documents = [
    tex_expr,
    another_tex_expr,
    r"\(\frac{1}{3} - x^{2} + \varphi(x(t))\)",
    r"\( \frac{1}{2} \)",
    r"\(f : \mathbb{R}^2 \to \mathbb{R}^3\)",
    r"\(\int \rho(x) dx\)",
    r"\(\alpha^2 + \beta^2 \neq \gamma^2\)",
]

Сформируем поисковый индекс, векторизовав все документы:

In [9]:
search_index = vectorizer.encode(documents)

Теперь создадим несколько поисковых запросов, использующих математические символы Unicode, разметку AsciiMath, транскрибацию формул и повреждённую разметку:

In [10]:
queries = [
    "∫ ρ(x) dx",
    "α^2 + β2 ≠ γ^2",
    "f : RR^(2) -> RR^(3)",
    "альфа + бета равно гамма",
    "интеграл ро от икс дэ икс",
     r"\( \frac{1{2 \)",
]

Теперь выполним поиск:

In [11]:
for query in queries:
    query_embedding = vectorizer.encode(query)
    similarity_scores = util.cos_sim(query_embedding, search_index)[0]
    scores, indices = torch.topk(similarity_scores, k=1)

    print("Запрос:", query)
    print("Лучший кандидат:", documents[indices[0]])
    print("Косинусное сходство:", scores[0].item())
    print()

Запрос: ∫ ρ(x) dx
Лучший кандидат: \(\int \rho(x) dx\)
Косинусное сходство: 0.9205648899078369

Запрос: α^2 + β2 ≠ γ^2
Лучший кандидат: \(\alpha^2 + \beta^2 \neq \gamma^2\)
Косинусное сходство: 0.9621996283531189

Запрос: f : RR^(2) -> RR^(3)
Лучший кандидат: \(f : \mathbb{R}^2 \to \mathbb{R}^3\)
Косинусное сходство: 0.9475846290588379

Запрос: альфа + бета равно гамма
Лучший кандидат: \(\alpha + \beta = \gamma\)
Косинусное сходство: 0.9391109943389893

Запрос: интеграл ро от икс дэ икс
Лучший кандидат: \(\int \rho(x) dx\)
Косинусное сходство: 0.7412841320037842

Запрос: \( \frac{1{2 \)
Лучший кандидат: \( \frac{1}{2} \)
Косинусное сходство: 0.996719479560852

